In [ ]:
import sys  
import math
from pysolar import solar
from datetime import datetime, timezone
from IPython.core.display import display, HTML, Markdown, Image


sys.path.insert(0, '../..')
sys.path.insert(0, '../helpers')

from deps.types import *
from deps.materials import * as materials
from sun_position import *

In [ ]:
class Structure:
    def __init__(self,
        width=6,
        depth=2.6,
        front_height=2.5,
        rear_height=3,
        barrel_count=4,
        plants_per_barrel=200,
        barrel_diameter=0.57,
        barrel_height=1.5,
        barrel_rotator_power=10,
        latitude=38.7436883,
        longitude=-9.1952227
        azimuth=180
    ):
        # Set up greenhouse primary parameters
        self.width: m = width
        self.depth: m = depth
        self.front_height: m = front_height
        self.rear_height: m = rear_height

        self.barrel_count = barrel_count
        self.plants_per_barrel = plants_per_barrel
        self.barrel_diameter: m = barrel_diameter
        self.barrel_height: m = barrel_height
        self.barrel_rotator_power: W = barrel_rotator_power

        # Set up location parameters
        self.coordinates = { "latitude": latitude, "longitude": longitude }
        self.azimuth = azimuth

        # Calculate derived parameters from primaries
        self.height_diff: m = self.rear_height - self.front_height
        self.roof_panel_depth: m = self.get_roof_panel_depth()
        self.roof_incline: deg = math.degrees(math.atan(self.height_diff / self.depth))

        self.barrel_circumference: m = math.pi * self.barrel_diameter
        self.barrel_surface_total: m2 = self.barrel_count * self.barrel_circumference * self.barrel_height
        self.barrel_surface_exposed_to_sun: m2 = self.barrel_surface_total / 3

        # Calculate surface areas (A) and volume (V)
        self.A_roof_panel: m2 = self.roof_panel_depth * self.width
        self.A_front_panel: m2 = self.width * self.front_height
        self.A_side_panel: m2 = self.get_side_panel_area()
        self.A_rear_panel: m2 = self.width * self.rear_height
        self.A_floor: m2 = self.width * self.depth
        self.volume: m3 = self.get_volume()

        self.panels = [
            {
                name: "slanted_roof",
                material: materials.Insolight(),
                area: self.A_roof_panel,
                tilt: self.roof_incline,
                azimuth_offset: 0
            },
            {
                name: "front_panel",
                material: materials.Polycarbonate(),
                area: self.A_front_panel,
                tilt: 90,
                azimuth_offset: 0
            },
            {
                name: "left_panel",
                material: materials.Polycarbonate(),
                area: self.A_side_panel,
                tilt: 90,
                azimuth_offset: 90
            },
            {
                name: "right_panel",
                material: materials.Polycarbonate(),
                area: self.A_side_panel,
                tilt: 90,
                azimuth_offset: -90
            },
            {
                name: "rear_panel",
                material: materials.Polycarbonate(),
                area: self.A_rear_panel
            },
            {
                name: "floor",
                material: materials.Plywood(),
                area: self.A_floor
            }
        ]


    def get_power_consumption(self) -> W:
        return self.barrel_count * self.barrel_rotator_power


    def get_roof_panel_depth(self) -> m:
        """
        Calculate depth (side edge if observing from front) of roof panel using Pythagoras' theorem
        """
        tan_alpha = self.height_diff / self.depth
        alpha: rad = math.atan(tan_alpha)
        return self.height_diff / math.sin(alpha)


    def get_side_panel_area(self) -> m2:
        A_triangle: m2 = (self.height_diff * self.depth) / 2
        A_rectangle: m2 = self.depth * self.front_height
        return A_triangle + A_rectangle


    def get_volume(self) -> m3:
        V_triangle: m3 = (self.height_diff * self.depth) / 2 * self.width
        V_rectangle: m3 = self.depth * self.front_height * self.width
        return V_triangle + V_rectangle


    def get_heat_transfer_rate(self, delta_T: C) -> W:
        heat_transfer_rate: W = 0
        for panel in self.panels:
            U: W_per_m2_K = panel["material"].U_value
            A: m2 = panel["area"]
            heat_transfer_rate_of_panel: W = U * A * delta_T
            heat_transfer_rate += heat_transfer_rate_of_panel

        return heat_transfer_rate


    def get_irradiance_in_greenhouse(timestamp, irradiance):
        irradiance_on_panels = {}
        for panel in panels:
            if "tilt" in panel:
                intensity_coeff_of_panel = get_intensity_coeff(self.coordinates, timestamp, panel["tilt"], self.azimuth + panel["azimuth_offset"])
                irradiance_on_panels[panel["name"]] = intensity_coeff_of_panel

        total_irradiance: W = 0
        for panel in irradiance_on_panels:
            total_irradiance += irradiance_on_panels[panel]

        return total_irradiance, irradiance_on_panels


### Test

In [ ]:
structure = Structure(
    width=10,
    depth=2,
    front_height=3,
    rear_height=4,
    barrel_count=4,
    plants_per_barrel=200,
    barrel_diameter=0.5,
    barrel_height=1.5,
    barrel_rotator_power=0
)

assert round(structure.roof_panel_depth, 3) == 2.236, "Error when calculating 'roof_panel_depth'"
assert round(structure.barrel_surface_total, 3) == 9.425, "Error when calculating 'barrel_surface_total'"
assert structure.volume == 70, "Error when calculating 'volume'"
assert round(structure.roof_incline, 3) == 26.565, "Error when calculating 'roof_incline'"



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>